In [1]:
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
import networkx as nx

In [2]:
data = pd.read_csv('NAS_processed_032523.csv')

## Clean up raw data and create network input dataframe

In [3]:
publications = data['Publication_Title']

In [4]:
publications_list = []

for v in publications:
    try:
        if float(v):
            publications_list.append(temp)
    except:
        temp = v
        publications_list.append(v)

In [5]:
#Capitalize first word in each publication title
publications_list_cap = []
for v in publications_list:
    first = v[0]
    publications_list_cap.append(first.upper()+v[1:])

In [6]:
Exposure_Layer3 = data['Exposure_Layer3']

In [7]:
Outcome_Layer3 = data['Outcome_Layer3']

In [8]:
hierach1 = defaultdict(set)
edge2publication = defaultdict(list)
for i,v in enumerate(Exposure_Layer3):
    hierach1[v].add(Outcome_Layer3[i])
    edge2publication[(v,Outcome_Layer3[i])].append(publications_list_cap[i])

In [9]:
intersection = set(Exposure_Layer3).intersection(set(Outcome_Layer3))

In [10]:
exposures_l3 = []
outcomes_l3 = []
exposure_cate = []
outcome_cate = []
edge_id2pub = dict()
i = 1
for k in sorted(hierach1.keys()): ###031823
    for v in sorted(hierach1[k]):
        exposures_l3.append(k)
        outcomes_l3.append(v)
        edge_id2pub[i] = edge2publication[(k,v)]
        i+=1
        if k not in intersection:
            exposure_cate.append('exposures')
        else:
            exposure_cate.append('both')
        if v not in intersection:
            outcome_cate.append('outcomes')
        else:
            outcome_cate.append('both')

In [11]:
edge_id = np.arange(1,len(edge_id2pub)+1).tolist()

In [12]:
layer3_dataframe = pd.DataFrame(list(zip(exposures_l3, outcomes_l3,edge_id,exposure_cate,outcome_cate)),columns =['Exposures', 'Outcomes','edge_id','exposure_cate','outcome_cate'])

In [13]:
layer3_dataframe

,Exposures,Outcomes,edge_id,exposure_cate,outcome_cate
0,APOE genetic variants,MMSE score,1,exposures,outcomes
1,BMI,high-density lipoprotein cholesterol,2,exposures,both
2,HFE gene,toenail cadmium,3,exposures,both
3,aging clocks,FEV1,4,both,outcomes
4,aging clocks,FEV1/FVC,5,both,outcomes
...,...,...,...,...,...
389,vitamin E intake,FEV1,390,exposures,outcomes
390,vitamin E intake,FVC,391,exposures,outcomes
391,western diet,blood lead,392,exposures,both
392,western diet,patella lead,393,exposures,both


In [14]:
pd.DataFrame.to_csv(layer3_dataframe,'processed_graph_layer3.csv')

## Create edge number to publication list

In [15]:
edg_list = []
pub_list = []
for k in range(1,len(edge_id2pub)+1):
    for v in set(edge_id2pub[k]):
        edg_list.append(k)
        pub_list.append(v)
    

In [16]:
layer3_publist = pd.DataFrame(list(zip(edg_list, pub_list)),columns =['Edge Id', 'Publication'])

In [17]:
layer3_publist

,Edge Id,Publication
0,1,Influence of multiple APOE genetic variants on...
1,2,Lifestyle change and high-density lipoprotein ...
2,3,"Iron-processing genotypes, nutrient intakes, a..."
3,4,Biomarkers of aging and lung function in the N...
4,5,Biomarkers of aging and lung function in the N...
...,...,...
423,390,Serum tocopherol levels and vitamin e intake a...
424,391,Serum tocopherol levels and vitamin e intake a...
425,392,A western diet pattern is associated with high...
426,393,A western diet pattern is associated with high...


In [18]:
pd.DataFrame.to_csv(layer3_publist,'layer3_publist.csv')